#RecipeFinder - Toidutoodete pildi põhjal retsepti soovitamine
Rühmaliikmed: Sander Põldma, Kristian Tamm

Kirjeldus: Lihtne programm, mis võtab sisendiks pildi, millel on selgelt näha erinevad toidud. Pildi põhjal tuvastab meie mudel olemasolevad toidutooted ja pakub sulle võimalikud retseptid.

##Plaan
*   Leiame kõige populaarsemad toiduained, mida meie programm võiks ära tunda
*   Genereerime pildid valitud toiduainetest, millest saab meie treening andmestik
*   Treenime Yolo mudelit toiduaineid tuvastama
*   Tuvastatud toiduainete põhjal retsepti soovitamine
*   Paneme kõik ühte kasutjasõbralikku Pythoni programmi



###Leiame populaarsed toiduained

In [1]:
# GD-ga ühendamine
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/Rühmatöö")
HOME = os.getcwd()
print(HOME)

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Rühmatöö


In [ ]:
#import
import pandas as pd
from collections import Counter
import ast
import pickle

In [ ]:
#https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
#Leiame kõige populaarsemad toiduained ja treenime hiljem enda mudelit neid ära tundma
data = pd.read_csv('RAW_recipes.csv')
raw_recipe_df = pd.DataFrame(data)

data = pd.read_csv('PP_recipes.csv')
recipe_df = pd.DataFrame(data)

ingredient_dict = pd.read_pickle('ingr_map.pkl')

In [ ]:
recipe_df.columns
recipe_df['ingredient_ids'] = recipe_df['ingredient_ids'].apply(ast.literal_eval)

In [ ]:
def is_subset_general(main_set, subset, percent):
    count = 0
    for element in subset:
        if element in main_set:
            count += 1
        else:
            for main_element in main_set:
                if element in main_element or main_element in element:
                    count += 1
    if count/len(subset) >= percent:
        return True
    return False

In [ ]:
#Lisame siia veel
target_ingredients = {'onion', 'egg', 'milk', 'garlic', 'cheese', 'lemon',
                         'carrot', 'tomato', 'chicken', 'celery',
                         'potato', 'parsley', 'bacon', 'red onion',
                         'ginger', 'apple', 'lettuce', 'pepper'}

# Initialize a list to store the selected rows
selected_rows = []

# Iterate through each row in the DataFrame
for index, row in recipe_df.sample(n=10000).iterrows():
    # Get the ingredients as a set
    ingredient_ids = row['ingredient_ids']
    ingredient_list = []

    for id in ingredient_ids:
        ingredient_list.append(ingredient_dict[ingredient_dict['id'] == id]['replaced'].iloc[0])

    ingredient_set = set(ingredient_list)

    if is_subset_general(target_ingredients, ingredient_set, 0.5):
        print('found row ----------------------------------------')
        selected_rows.append(row)

    print('row' + str(index))

sorted_recipe_df = pd.DataFrame(selected_rows)
sorted_recipe_df.to_csv('edited_recipes.csv', index=False)

Voogesituse väljund on kärbitud viimase 5000 reani.
found row ----------------------------------------
row2953
row97070
row46734
row170673
row148810
row176231
found row ----------------------------------------
row167727
found row ----------------------------------------
row5508
row17508
row50552
row90991
row15403
row67476
found row ----------------------------------------
row76079
found row ----------------------------------------
row78187
row102483
row44605
found row ----------------------------------------
row12563
row169851
row99568
row46493
row29073
found row ----------------------------------------
row52654
row108989
found row ----------------------------------------
row70553
row138494
row62160
row25463
row150669
found row ----------------------------------------
row123204
found row ----------------------------------------
row158321
row108174
row135412
found row ----------------------------------------
row33261
row85548
row112025
row129202
row134610
found row ---------------------

In [ ]:
all_ingredients = []
for ingredients in recipe_df['ingredient_ids']:
  for ingredient in ingredients:
    all_ingredients.append(ingredient)

counter = Counter(all_ingredients)
top_ids = [ingredient_id for ingredient_id, _ in counter.most_common(100)]
print(top_ids)

[6270, 840, 5010, 2499, 6906, 5006, 3203, 7655, 4717, 2683, 5319, 800, 3184, 332, 6335, 6276, 63, 1168, 5180, 335, 4253, 7449, 7557, 1093, 6654, 590, 1511, 7213, 6696, 1910, 3217, 1257, 4623, 4987, 1252, 1124, 3723, 7470, 5648, 7946, 5168, 2856, 1833, 5185, 298, 5966, 7367, 1329, 2816, 3497, 4836, 4964, 3484, 1119, 7642, 3502, 5298, 2148, 4574, 3440, 4308, 3355, 5915, 1388, 7258, 6324, 2832, 393, 5695, 4863, 2131, 342, 2320, 8021, 7233, 4096, 2509, 5825, 5058, 3668, 2200, 1043, 2809, 5106, 2777, 1297, 869, 4053, 2007, 6426, 1786, 3399, 7229, 150, 339, 4231, 3486, 2757, 4384, 3512]


In [ ]:
#Vaatame mis toiduaine peitub iga id taga
top_toiduained = []
for id in top_ids:
  top_toiduained.append(ingredient_dict[ingredient_dict['id'] == id]['replaced'].iloc[0])

print(top_toiduained)

['salt', 'butter', 'onion', 'egg', 'sugar', 'olive oil', 'garlic clove', 'water', 'milk', 'flmy', 'pepper', 'brown sugar', 'garlic', 'baking powder', 'scallion', 'salt and pepper', 'all-purpose flmy', 'cheddar', 'parmesan cheese', 'baking soda', 'lemon juice', 'vanilla', 'vegetable oil', 'carrot', 'sour cream', 'black pepper', 'cinnamon', 'tomato', 'soy sauce', 'cream cheese', 'garlic powder', 'chicken broth', 'mayonnaise', 'oil', 'chicken breast', 'celery', 'honey', 'vanilla extract', 'potato', 'worcestershire sauce', 'paprika', 'fresh parsley', 'cornstarch', 'parsley', 'bacon', 'red onion', 'unsalted butter', 'chili powder', 'fresh ground black pepper', 'ground cinnamon', 'mozzarella', 'nutmeg', 'ground beef', 'cayenne pepper', 'walnut', 'ground cumin', 'pecan', 'dijon mustard', 'margarine', 'green pepper', 'lettuce', 'granulated sugar', 'red bell pepper', 'chocolate chip', 'tortilla', 'sausage', 'fresh lemon juice', 'bay leaf', 'powdered sugar', 'mushroom', 'diced tomato', 'banana',

In [ ]:
#Lisame listi toiduained, mis enamus köökides alati olemas on ja mida oleks võimalik suuremate raskusteta mudelil ära tunda
final_ingredient_list = ['onion', 'egg', 'milk', 'garlic', 'cheese', 'lemon',
                         'carrot', 'tomato', 'chicken breast', 'celery',
                         'potato', 'parsley', 'bacon', 'red onion',
                         'ginger', 'apple', 'lettuce', 'pepper']
len(final_ingredient_list)

18

###Dataset V1 ja Yolov9 treenimine

In [ ]:
#Impordime oma dataseti
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="nHiCIDzulosCIHBDRHFC")
project = rf.workspace("aiproject-vf2oo").project("food-detector-rmlfo")
version = project.version(4)
dataset = version.download("yolov9")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Food-detector-4 in yolov9:: 100%|██████████| 5315/5315 [00:43<00:00, 122.11it/s]


In [ ]:
!pip -q install ultralytics roboflow supervision

ERROR: Operation cancelled by user


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov9c.yaml")

model = YOLO("yolov9c.pt")

model.info()

In [ ]:
results = model.train(data="/content/drive/MyDrive/Colab Notebooks/Rühmatöö/Food-detector-4/data.yaml", epochs=20, imgsz=640)

###Implementeerime treenitud mudeli

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

  Using cached ultralytics-8.2.22-py3-none-any.whl (778 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)


In [ ]:
model = YOLO("runs/detect/train7/weights/best.pt")

In [ ]:
from PIL import Image

results = model("test_images/test3_con.jpg", save_txt=True)
#for r in results:
    #im_array = r.plot()  # plot a BGR numpy array of predictions
    #im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    #im.show()  # show image
    #im.save('results9.jpg')


image 1/1 /content/drive/MyDrive/Colab Notebooks/Rühmatöö/test_images/test3_con.jpg: 480x640 1 garlic, 2 lemons, 2 onions, 1 potato, 1 tomato, 1646.2ms
Speed: 5.8ms preprocess, 1646.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels
